# Running this notebook

**The visualizations in this notebook will run in [jupyter lab](https://github.com/jupyterlab/jupyterlab#installation), not jupyter notebook. Google colab is not supported either. VS Code notebooks _might_ work but that has not been tested.** See the fastplotlib supported frameworks for more info: https://github.com/fastplotlib/fastplotlib/#supported-frameworks 

In [1]:
from copy import deepcopy
from pathlib import Path
import os

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import fastplotlib as fpl

import mesmerize_core as mc
import mesmerize_viz

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

# Mac users!
# temporary patch for Mac, won't be necessary in next release
# Thanks Ryan Ly for the PR! :D I need to dig into it more before merging
# conda_prefix_1_str = os.environ['CONDA_PREFIX'].replace(os.path.join(' ', 'envs', 'mescore')[1:], '')
# os.environ['CONDA_PREFIX_1'] = conda_prefix_1_str

# This is just a pandas table display formatting option
pd.options.display.max_colwidth = 120

home_path = Path(os.environ.get("HOME", Path().home()))
f"Home Path: {home_path}"

2024-08-17 19:31:01.340462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 19:31:01.340494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 19:31:01.340871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 19:31:01.344354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'Home Path: /home/mbo'

In [2]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

raw_tiff_path = home_path / 'caiman_data' / 'high_res' / 'final'
raw_tiff_name = home_path / 'extracted_plane_1.tif'

mc.set_parent_raw_data_path(raw_tiff_path.expanduser())
print(f"Files: {[str(x) for x in raw_tiff_path.glob('*.tif*')]}")

Files: ['/home/mbo/caiman_data/high_res/final/extracted_plane_1.tif']


In [3]:
!ls -la /home/mbo/caiman_data/high_res/final/batch

Sourcing .zshenv...
total 60
drwxr-xr-x 4 mbo mbo 4096 Aug 17 18:19 .
drwxr-xr-x 4 mbo mbo 4096 Aug 16 11:44 ..
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:49 07e78cfc-4f38-4bc0-aafb-5ba718f65406.runfile
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:45 262786b6-857b-4c67-a4e4-10912567920b.runfile
drwxr-xr-x 2 mbo mbo 4096 Aug 16 14:05 455f1a04-d06e-447b-b73f-64ff9affd62d
-rwxr--r-- 1 mbo mbo  457 Aug 16 14:04 455f1a04-d06e-447b-b73f-64ff9affd62d.runfile
drwxr-xr-x 2 mbo mbo 4096 Aug 16 11:45 56dfceb9-059b-46c7-a656-f0bea5d93b65
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:45 56dfceb9-059b-46c7-a656-f0bea5d93b65.runfile
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:47 639ca972-9bd3-4373-8553-1818f123bb6f.runfile
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:48 8a21cd28-8f0a-4796-8847-8f026daedc47.runfile
-rwxr--r-- 1 mbo mbo  457 Aug 16 11:47 c8d10552-3171-45b9-9ea7-a6752bf24029.runfile
-rw-r--r-- 1 mbo mbo 2015 Aug 16 14:05 cnmf
-rwxr--r-- 1 mbo mbo  456 Aug 16 13:52 f2f5479a-d1b9-4d95-bf1c-cef979f67cf0.runfile
-rwxr--r-- 1 mbo mbo

# Paths

`mesmerize-core` helps manage the outputs of caiman algorithms and organizes "parameter variants" - the output of a given combination of input data and algorithm parameters. In order to run the algorithms you must tell `mesmerize-core` where your _input data_ are located and decide on a **top level raw data directory**. For example consider the following directory structure of experimental data (you may organize your raw data however you wish, this is just an example). We can see that all the experimental data lies under `/data/group_name/my_name/exp_data`. Therefore we can use this `exp_data` dir as a `parent raw data path`. `mesmerize-core` will then only store the _relative_ paths to the raw data files, this allows you to move datasets between computers and filesystems. `mesmerize-core` does not store any hard file paths, only relative paths.

```
/data/group_name/my_name
                        └── exp_data
                            ├── axonal_imaging
                            │   ├── mouse_1
                            │   │   ├── exp_a.tiff
                            │   │   ├── exp_b.tiff
                            │   │   └── exp_c.tiff
                            │   ├── mouse_2
                            │   │   ├── exp_a.tiff
                            │   │   └── exp_b.tiff
                            │   └── mouse_3
                            └── hippocampus_imaging
                                ├── mouse_1
                                │   ├── exp_a.tiff
                                │   ├── exp_b.tiff
                                │   └── exp_c.tiff
                                ├── mouse_2
                                └── mouse_3
```

**For this demo set the `caiman_data` dir as the parent raw data path**

Sidenote: We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems. Therefore even if you are on Windows you can use the regular `/` for paths, you do not have to worry about the strangeness of `\\` and `\`

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

In [8]:
batch_path = mc.get_parent_raw_data_path().joinpath("batch/reg")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [9]:
# create a new batch
#df = mc.create_batch(batch_path)

# to load existing batches use `load_batch()`
df = mc.load_batch(batch_path)

# View the dataframe

It is empty and has the required columns for mesmerize

In [10]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (4, 4), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 56dfceb9-059b-46c7-a656-f0bea5d93b65/56dfceb9-059b-46c7-a656-f0bea5d93b65_mean_projection.n...,2024-08-16T11:45:16,2024-08-16T11:45:54,22.55 sec,None,56dfceb9-059b-46c7-a656-f0bea5d93b65


# Let's add stuff to the DataFrame!

First get an input movie. An input movie must be somewhere under `parent raw data path`. It does not have to be directly under `parent raw data path`, it can be deeply nested anywhere under it.

# Motion correction parameters

The parameters are passed **directly** to `caiman`, this means you need to use the same exact names for the parameters and you can use all the parameters that you can use with `caiman` - because it's just passing them to `caiman`.


The parameters dict for a mesmerize batch item must have the following structure. Put all the parameters in a dict under a key called **main**. The **main** dict is then fed directly to `caiman`.

```python
{"main": {... params directly passed to caiman}}
```

In [13]:
# We will start with one version of parameters
mcorr_params =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [20, 20],
        'strides': [64, 64],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
    },
}

# Add a "batch item" to the DataFrame this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

We can now see that there is one item in the DataFrame. What we called a "item" in `mesmerize-core` DataFrames is technically called a pandas `Series` or row.

# Run an item

There is only one item in this DataFrame and it is located at index `0`. You can run a row using `df.iloc[index].caiman.run()`

Technical notes: On Linux & Mac it will run in subprocess but on Windows it will run in the local kernel.

In [ ]:
df.iloc[0].caiman.run()

# Reload DataFrame from disk

After running one or any number of items in `mesmerize-core` you must call `df = df.caiman.reload_from_disk()`. This loads the DataFrame with references to the output files in the batch directory.

In [44]:
df = df.caiman.reload_from_disk()

In [45]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (4, 4), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 56dfceb9-059b-46c7-a656-f0bea5d93b65/56dfceb9-059b-46c7-a656-f0bea5d93b65_mean_projection.n...,2024-08-16T11:45:16,2024-08-16T11:45:54,22.55 sec,None,56dfceb9-059b-46c7-a656-f0bea5d93b65
1,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (2, 2), 'strides': (12, 12), 'overlaps': (6, 6), 'max_deviation_rigid': 3, 'border_nan': 'co...",{'mean-projection-path': 262786b6-857b-4c67-a4e4-10912567920b/262786b6-857b-4c67-a4e4-10912567920b_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:46:48,50.65 sec,None,262786b6-857b-4c67-a4e4-10912567920b
2,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (2, 2), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': f9dea974-9356-41b2-b3e4-288cda41e860/f9dea974-9356-41b2-b3e4-288cda41e860_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:47:19,28.08 sec,None,f9dea974-9356-41b2-b3e4-288cda41e860
3,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (2, 2), 'strides': (64, 64), 'overlaps': (32, 32), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': c8d10552-3171-45b9-9ea7-a6752bf24029/c8d10552-3171-45b9-9ea7-a6752bf24029_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:47:46,22.99 sec,None,c8d10552-3171-45b9-9ea7-a6752bf24029
4,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (32, 32), 'strides': (12, 12), 'overlaps': (6, 6), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 639ca972-9bd3-4373-8553-1818f123bb6f/639ca972-9bd3-4373-8553-1818f123bb6f_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:48:39,50.48 sec,None,639ca972-9bd3-4373-8553-1818f123bb6f
5,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (32, 32), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 8a21cd28-8f0a-4796-8847-8f026daedc47/8a21cd28-8f0a-4796-8847-8f026daedc47_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:49:11,28.62 sec,None,8a21cd28-8f0a-4796-8847-8f026daedc47
6,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (32, 32), 'strides': (64, 64), 'overlaps': (32, 32), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 07e78cfc-4f38-4bc0-aafb-5ba718f65406/07e78cfc-4f38-4bc0-aafb-5ba718f65406_mean_projection.n...,2024-08-16T11:45:24,2024-08-16T11:49:38,23.19 sec,None,07e78cfc-4f38-4bc0-aafb-5ba718f65406
7,cnmf,extracted_plane_1,07e78cfc-4f38-4bc0-aafb-5ba718f65406/07e78cfc-4f38-4bc0-aafb-5ba718f65406-extracted_plane_1_els__d1_583_d2_544_d3_1_...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...","{'success': False, 'traceback': 'Traceback (most recent call last): File ""/home/mbo/miniconda3/envs/mescore/lib/py...",2024-08-16T13:52:01,2024-08-16T13:53:21,49.73 sec,None,f2f5479a-d1b9-4d95-bf1c-cef979f67cf0


# Outputs

We can see that the outputs column has been populated. The entries in this column do not have to be accessed directly. The `mesmerize-core` API allows you to fetch these outputs.

In [46]:
index = 0 # we will fetch stuff from index 0 which we just ran

# get the motion corrected movie memmap
mcorr_movie = df.iloc[0].mcorr.get_output()
mcorr_movie.shape

(800, 583, 544)

In [47]:
# path to the mcorr memmap if you ever need it
mcorr_memmap_path = df.iloc[0].mcorr.get_output_path()
mcorr_memmap_path

PosixPath('/home/mbo/caiman_data/high_res/final/batch/56dfceb9-059b-46c7-a656-f0bea5d93b65/56dfceb9-059b-46c7-a656-f0bea5d93b65-extracted_plane_1_els__d1_583_d2_544_d3_1_order_F_frames_800.mmap')

In [48]:
# mean projection, max and std projections are also available
mean_proj = df.iloc[0].caiman.get_projection("mean")
mean_proj.shape

(583, 544)

In [49]:
# the input movie, note that we use `.caiman` here instead of `.mcorr`
input_movie = df.iloc[0].caiman.get_input_movie()
input_movie.shape

(800, 583, 544)

In [62]:
df.drop(0, inplace=True)

## Note on input movies

`get_input_movie()` will automatically work with most tiff files and memmaps. If you want to load other file types, you will need to pass it a function (see examples below) that returns a lazy-loadable array, or a numpy array if you have enough RAM. 


### tiff files

`get_input_movie()` wil try to use `tifffile.memmap` to lazy-load tiff files. This works for some tiff files. If `tifffile.memmap` fails, `mesmerize-core` has its own `LazyTiff` implementation that it will try to fallback on. However, some not every tiff file can be lazy-loaded so it is not gaurenteed that `LazyTiff` will work. The implementation of `LazyTiff` is quite simple and you might be able to subclass it for your specific type of tiff file: https://github.com/nel-lab/mesmerize-core/blob/master/mesmerize_core/arrays/_tiff.py 

### hdf5 files

```python
import h5py

def hdf5_reader(path):
    f = h5py.File(path)
    return f["your-key"]

input_movie = df.iloc[index].caiman.get_input_movie(hdf5_reader)
```

### avi files
    
```python
from mesmerize_core.arrays import LazyVideo

def video_reader(path):
    a = LazyVideo(path)  # you can use the other args if you want
    return a

input_movie = df.iloc[index].caiman.get_input_movie(video_reader)
```

# Visualize with mesmerize-viz!

- Random-access frames
- Slider for frame averaging over a window - useful for quality control
- adjust cmap, vmin vmax, etc.

In [43]:
viz = df.mcorr.viz()
viz.show()

/home/mbo/miniconda3/envs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:512: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/mbo/miniconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [53]:
viz.close()

# Visualizations are customizable

Hint: use `Shift` + `Tab` to bring up the docstring for `mcorr.viz()`.

In [15]:
viz = df.mcorr.viz(
    data_options=["input", "mcorr"],
    image_widget_kwargs={"grid_plot_kwargs": {"size": (800, 400)}},
)
viz.show(sidecar=True)  # show in sidecar

/home/mbo/miniconda3/envs/mestest/lib/python3.11/site-packages/ipydatagrid/datagrid.py:512: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/mbo/miniconda3/envs/mestest/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


AttributeError: 'GlyphSlot' object has no attribute 'render'

# Customization

We call `viz` a "Viz Container". The `McorrVizContainer` has a fastplotlib `ImageWidget` that you can access.

In [21]:
viz

**Use tab completion to see public methods and attributes**

``viz.<Tab>``

In [22]:
viz.image_widget

Change colormaps

In [23]:
viz.image_widget.cmap = "gray"

In [ ]:
viz.image_widget.cmap = "viridis"

Access subplots, graphics, etc. The full suite of the `fastplotlib` API

In [ ]:
viz.image_widget.gridplot["mcorr"]

In [ ]:
viz.image_widget.gridplot["mcorr"]["image_widget_managed"].cmap.vmax

In [24]:
viz.image_widget.gridplot["mcorr"]["image_widget_managed"].cmap.vmax = 250

# Close visualization when we are done

In [25]:
viz.close()

# Parameter variants - this is the purpose of mesmerize-core!

Let's add another row to the DataFrame. We will use the same input movie but different parameters. This is the basis of how we can perform a _parameter gridsearch_.

In [ ]:
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [4, 4],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

We can see that there are two batch items for the same input movie.

# Parameter Gridsearch

Use a `for` loop to add multiple different parameter variants more efficiently.

In [ ]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [2, 32]: 
    for strides in [12, 24, 64]:
        overlaps = int(strides / 2)
        # deep copy is the safest way to copy dicts
        new_params = deepcopy(new_params)

        # assign the "max_shifts"
        new_params["main"]["max_shifts"] = (shifts, shifts)
        new_params["main"]["strides"] = (strides, strides)
        new_params["main"]["overlaps"] = (overlaps, overlaps)

        df.caiman.add_item(
          algo='mcorr',
          item_name=movie_path.stem,
          input_movie_path=movie_path,
          params=new_params
        )

In [ ]:
df

# Distinguishing parameter variants

We can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

We can use the `caiman.get_params_diffs()` to see the unique parameters between rows with the same `item_name`

In [42]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

""
0


# Run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Outputs

Load the output information into the DataFrame

In [38]:
df = df.caiman.reload_from_disk()

In [39]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (4, 4), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 56dfceb9-059b-46c7-a656-f0bea5d93b65/56dfceb9-059b-46c7-a656-f0bea5d93b65_mean_projection.n...,2024-08-16T11:45:16,2024-08-16T11:45:54,22.55 sec,None,56dfceb9-059b-46c7-a656-f0bea5d93b65


# Visualization using `mesmerize-viz` 

In [40]:
mcorr_viz = df.mcorr.viz(
    data_options=["input", "mcorr"], 
    image_widget_kwargs={"grid_plot_kwargs": {"size": (1000, 500)}}  # you can also pass kwargs to the ImageWidget that is created
)
mcorr_viz.show()

/home/mbo/miniconda3/envs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:512: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/mbo/miniconda3/envs/mescore/lib/python3.11/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [41]:
# close when done
mcorr_viz.close()

# Build your own visualizations using `fastplotlib`

# Use `ImageWidget` to view multiple mcorr results simultaneously

This type of visualization usually requires your files to be lazy-loadble, and the performance will depend on your hard drive's capabilities.

In [37]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie()]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# get the param diffs to set plot titles
param_diffs = df.caiman.get_params_diffs("mcorr", item_name=df.iloc[0]["item_name"])

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())

    max_shifts = param_diffs.iloc[i]["max_shifts"][0]
    strides = param_diffs.iloc[i]["strides"][0]
    overlaps = param_diffs.iloc[i]["overlaps"][0]
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix {i}: max_sh: {max_shifts}, str: {strides}, ove: {overlaps}")
    
    # mean images which we'll use later
    means.append(row.caiman.get_projection("mean"))

# create the widget
mcorr_iw_multiple = fpl.ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 17)}, # window functions as a kwarg, this is what the slider was used for in the ready-made viz
    grid_plot_kwargs={"size": (900, 700)},
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)

mcorr_iw_multiple.show()

KeyError: 'max_shifts'

Optionally hide the histogram LUT tool

In [ ]:
for subplot in mcorr_iw_multiple.gridplot:
    subplot.docks["right"].size = 0

Modify the `window_funcs` at any time. This is what the slider in `mesmerize-viz` does.

In [ ]:
mcorr_iw_multiple.window_funcs["t"].window_size = 43

In [ ]:
mcorr_iw_multiple.close()

# Optional, cleanup DataFrame

ix `6` seems to work the best so we will cleanup the DataFrame and remove all other items.

Remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`. This also cleans up the output data in the batch directory.

In [34]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[0].uuid]
rows_keep

['56dfceb9-059b-46c7-a656-f0bea5d93b65']

**Note:** On windows calling `remove_item()` will raise a `PermissionError` if you have the memmap file open. The workaround is to shutdown the current kernel and then use `df.caiman.remove_item()`. For example, you can keep another notebook that you use just for cleaning unwanted mcorr items.

There is currently no way to close a `numpy.memmap`: https://github.com/numpy/numpy/issues/13510

In [36]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid, safe_removal=False)

df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,extracted_plane_1.tif,"{'main': {'max_shifts': (4, 4), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 56dfceb9-059b-46c7-a656-f0bea5d93b65/56dfceb9-059b-46c7-a656-f0bea5d93b65_mean_projection.n...,2024-08-16T11:45:16,2024-08-16T11:45:54,22.55 sec,None,56dfceb9-059b-46c7-a656-f0bea5d93b65


Indices are always reset when you use `caiman.remove_item()`. UUIDs are always preserved.

# CNMF

Perform CNMF using the mcorr output.

Similar to mcorr, put the CNMF params within the `main` key. The `refit` key will perform a second iteration, as shown in the `caiman` `demo_pipeline.ipynb` notebook.

In [3]:
# some params for CNMF
params_cnmf = {
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

# Add CNMF item

You can provide the mcorr item row to `input_movie_path` and it will resolve the path of the input movie from the entry in the DataFrame.

In [ ]:
good_mcorr_index = 6

# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[good_mcorr_index],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[good_mcorr_index]["item_name"], # use the same item name
)

See the cnmf item at the bottom of the dataframe

In [ ]:
df

# Run CNMF

The API is identical to running mcorr

In [ ]:
index = -1  # most recently added item
df.iloc[index].caiman.run()

# Reload dataframe

In [ ]:
df = df.caiman.reload_from_disk()
df

# CNMF outputs

Similar to mcorr, you can use the `mesmerize-core` API to fetch the outputs. The API reference for CNMF is here: https://mesmerize-core.readthedocs.io/en/latest/api/cnmf.html

In [ ]:
index = -1  # index of the cnmf item, last item in the dataframe

# temporal components
temporal = df.iloc[index].cnmf.get_temporal()

In [ ]:
temporal.shape

Many of the cnmf functions take a rich set of arguments

In [ ]:
# get accepted or rejected components
temporal_good = df.iloc[index].cnmf.get_temporal("good")

# shape is [n_components, n_frames]
temporal_good.shape

In [ ]:
# get specific components
df.iloc[index].cnmf.get_temporal(np.array([1, 5, 9]))

In [ ]:
# get temporal with the residuals, i.e. C + YrA
temporal_with_residuals = df.iloc[index].cnmf.get_temporal(add_residuals=True)

In [ ]:
# get contours
contours = df.iloc[index].cnmf.get_contours()

Returns: `(list of np.ndarray of contour coordinates, list of center of mass)`

In [ ]:
print(f"contour 0 coordinates:\n\n{contours[0][0]}\n\n com: {contours[1][0]}")

In [ ]:
len(contours)

In [ ]:
# get_contours() also takes arguments
contours_good = df.iloc[index].cnmf.get_contours("good")

In [ ]:
len(contours_good[0]) # number of contours

swap_dim

In [ ]:
# get the first contour using swap_dim=True (default)
swap_dim_true = df.iloc[index].cnmf.get_contours()[0][0]

In [ ]:
# get the first contour  with swap_dim=False
swap_dim_false = df.iloc[index].cnmf.get_contours(swap_dim=False)[0][0]

In [ ]:
plt.plot(
    swap_dim_true[:, 0], 
    swap_dim_true[:, 1], 
    label="swap_dim=True"
)
plt.plot(
    swap_dim_false[:, 0], 
    swap_dim_false[:, 1], 
    label="swap_dim=False"
)
plt.legend()

In [ ]:
# swap_dim swaps the x and y dims
plt.plot(
    swap_dim_true[:, 0], 
    swap_dim_true[:, 1], 
    linewidth=30
)
plt.plot(
    swap_dim_false[:, 1], 
    swap_dim_false[:, 0], 
    linewidth=10
)

# Reconstructed movie - `A * C`
# Reconstructed background - `b * f`
# Residuals - `Y - AC - bf` 

Mesmerize-core provides these outputs as lazy arrays. This allows you to work with arrays that would otherwise be hundreds of gigabytes or terabytes in size.

In [ ]:
rcm = df.iloc[index].cnmf.get_rcm()
rcm

LazyArrays behave like numpy arrays

In [ ]:
rcm[42]

In [ ]:
rcm[10:20].shape

# Using LazyArrays

In [ ]:
rcm_accepted = df.iloc[index].cnmf.get_rcm("good")
rcm_rejected = df.iloc[index].cnmf.get_rcm("bad")

In [ ]:
iw_max = fpl.ImageWidget(
    data=[rcm_accepted.max_image, rcm_rejected.max_image],
    names=["accepted", "rejected"],
    grid_plot_kwargs={"size": (900, 450)},
    cmap="gnuplot2"
)
iw_max.show()

In [ ]:
iw_rcm_separated = fpl.ImageWidget(
    data=[rcm_accepted, rcm_rejected],
    names=["accepted", "rejected"],
    grid_plot_kwargs={"size": (900, 450)},
    cmap="gnuplot2"
)
iw_rcm_separated.show()

# All CNMF LazyArrays

In [ ]:
rcb = df.iloc[index].cnmf.get_rcb()
residuals = df.iloc[index].cnmf.get_residuals()
input_movie = df.iloc[index].caiman.get_input_movie()

`ImageWidget` accepts arrays that are sufficiently numpy-like

In [ ]:
iw_rcm = fpl.ImageWidget(
    data=[input_movie, rcm, rcb, residuals], 
    grid_plot_kwargs={"size": (800, 600)}, 
    cmap="gnuplot2"
)
iw_rcm.show()

In [ ]:
iw_rcm.close()

# Visualize everything with `mesmerize-viz`

In [ ]:
viz_cnmf = df.cnmf.viz()
viz_cnmf.show()

In [ ]:
viz_cnmf.close()

# Parameter Gridsearch

As shown for motion correction, the purpose of `mesmerize-core` is to perform parameter searches

In [ ]:
# itertools.product makes it easy to loop through parameter variants
# basically, it's easier to read that deeply nested for loops
from itertools import product

# variants of several parameters
gSig_variants = [4, 6]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[good_mcorr_index]["item_name"],  # good mcorr item
        input_movie_path=df.iloc[good_mcorr_index],
        params=new_params_cnmf
    )

We now have lot of cnmf items

In [ ]:
df

View the diffs

In [ ]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[-1]["item_name"])

# Run the `cnmf` batch items

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Load outputs

In [ ]:
df = df.caiman.reload_from_disk()

In [ ]:
df

# Visualize with `mesmerize-viz`

In [ ]:
viz_cnmf = df.cnmf.viz()
viz_cnmf.show(sidecar=True)

# Caiman docs on component eval

https://caiman.readthedocs.io/en/latest/Getting_Started.html#component-evaluation

> The quality of detected components is evaluated with three parameters:
>
> Spatial footprint consistency (rval): The spatial footprint of the component is compared with the frames where this component is active. Other component’s signals are subtracted from these frames, and the resulting raw data is correlated against the spatial component. This ensures that the raw data at the spatial footprint aligns with the extracted trace.
>
> Trace signal-noise-ratio (SNR): Peak SNR is calculated from strong calcium transients and the noise estimate.
>
> CNN-based classifier (cnn): The shape of components is evaluated by a 4-layered convolutional neural network trained on a manually annotated dataset. The CNN assigns a value of 0-1 to each component depending on its resemblance to a neuronal soma.
> 
> Each parameter has a low threshold:
> - (rval_lowest (default -1), SNR_lowest (default 0.5), cnn_lowest (default 0.1))
>
> and high threshold
> 
> - (rval_thr (default 0.8), min_SNR (default 2.5), min_cnn_thr (default 0.9))
> 
> A component has to exceed ALL low thresholds as well as ONE high threshold to be accepted.

# This rich visualization is still customizable!

Public attributes:

- `image_widget`: the `ImageWidget` in the visualization
- `plot_temporal`: the `Plot` with the temporal
- `plot_heatmap`: the `Plot` with the heatmap
- `cnmf_obj`: The cnmf object currently being visualized. This object gets saved to disk when you click the "Save Eval to disk" button.
- `component_index`: current component index, `int`

A few public methods:
- `show()` show the visualization
- `set_component_index(index: int)` manually set the component index

In [ ]:
viz_cnmf.image_widget.cmap = "gray"

In [ ]:
viz_cnmf.plot_heatmap

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap = "viridis"

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap.vmax

In [ ]:
viz_cnmf.plot_heatmap["heatmap"].cmap.vmax = 2_000

Customize contours

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"][:].thickness = 1.0

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"].visible = False

In [ ]:
for subplot in viz_cnmf.image_widget.gridplot:
    subplot["contours"].visible = True

In [ ]:
viz_cnmf.plot_temporal["line"].thickness()

In [ ]:
viz_cnmf.plot_temporal["line"].thickness = 1

# Visualize fewer things

In [ ]:
viz_simple = df.cnmf.viz(
    image_data_options=["input", "rcm"],
)
viz_simple.show(sidecar=True)

# More customization of kwargs

In [ ]:
viz_more_custom = df.cnmf.viz(
    image_data_options=["input", "rcm", "rcm-max", "corr"],
    temporal_kwargs={"add_residuals": True},
)

In [ ]:
viz_more_custom.show(sidecar=True)